In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from torch import nn
from torch.nn import functional as F
import torch
import pickle
from keras.preprocessing import sequence
import string
import re

In [0]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [0]:
def clean_special_chars(data):
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
    def func(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text

    return func(data, punct)


def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")

    for key in mapping.keys():
      if key in text:
        text = text.replace(key, mapping[key])

    text = [word.replace("'s", "") if word.endswith("'s") else word for word in text.split()]
    text = ' '.join(text)
    return text


def replace_repeating_chars(text):
    for char in string.ascii_lowercase:
      pattern = char + '{2,}'
      text = re.sub(pattern, char, text)
    return text


def preprocess(data):
  data = data.lower()
  data = clean_contractions(data, contraction_mapping)
  data = clean_special_chars(data)
  #data = replace_repeating_chars(data)
  return data

In [0]:
LSTM_UNITS = 64
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS

class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    
        x = x.permute(0, 3, 2, 1)  
        x = super(SpatialDropout, self).forward(x)  
        x = x.permute(0, 3, 2, 1)  
        x = x.squeeze(2)  
        return x

class ToxicLSTM(nn.Module):
  
    def __init__(self, n_targets, max_features, embed_size):
      
        super(ToxicLSTM, self).__init__()
        
        self.embedding = nn.Embedding(max_features+1, embed_size)
        self.embedding_dropout = SpatialDropout(0.3)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)
    
        self.linear = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
     
        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)
        self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, n_targets)
        
        
    def forward(self, x):
      
        m_embed = self.embedding(x)  
        m_embed = self.embedding_dropout(m_embed)
        
        m_lstm, _ = self.lstm1(m_embed)
        m_lstm, _ = self.lstm2(m_lstm)
        
        avg_pool = torch.mean(m_lstm, 1)
        max_pool, _ = torch.max(m_lstm, 1)
        m_conc = torch.cat((max_pool, avg_pool), 1) 

        m_conc_linear  = F.relu(self.linear(m_conc))

        hidden = m_conc + m_conc_linear
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)      

        return out

In [0]:
tok_path = F"/content/gdrive/My Drive/pytorch/tokenizer.pickle"
with open(tok_path, 'rb') as f:
  tokenizer = pickle.load(f)

In [11]:
max_features = len(tokenizer.word_index) 
print('max_features: {}'.format(max_features))

max_features: 150000


In [14]:
model_path = F"/content/gdrive/My Drive/pytorch/model_lstm_state_dict.pt"
state_dict = torch.load(model_path, map_location='cpu')
model = ToxicLSTM(6, max_features, 300)
model.load_state_dict(state_dict['state_dict'])
model.eval()

ToxicLSTM(
  (embedding): Embedding(150001, 300)
  (embedding_dropout): SpatialDropout(p=0.3)
  (lstm1): LSTM(300, 64, batch_first=True, bidirectional=True)
  (lstm2): LSTM(128, 64, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=256, out_features=256, bias=True)
  (linear_out): Linear(in_features=256, out_features=1, bias=True)
  (linear_aux_out): Linear(in_features=256, out_features=6, bias=True)
)

In [15]:
state_dict['roc_auc']

0.9610548277648558

In [0]:
def getPredictions(comments, model):
  comments = list(map(preprocess, comments))
  comments = tokenizer.texts_to_sequences(comments)
  MAX_LEN = max([len(x) for x in comments])
  comments = sequence.pad_sequences(comments, maxlen=MAX_LEN)
  comments = torch.tensor(comments, dtype=torch.long)
  results = F.sigmoid(model(comments)).tolist()
  return results
  

In [0]:
comments = ['I will kill you', 'Proud member of LGBT community']

In [26]:
preds = getPredictions(comments, model)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
